In [1]:
import pandas as pd
import numpy as np
import math
from functools import partial, reduce
import os
import re

In [2]:
dfs = []
input_path = 'C:/Users/lzoeckler/Desktop/menzies_raw'
for fname in os.listdir(input_path):
    plex_data = pd.read_csv('{}/{}'.format(input_path, fname),
                            skiprows=13, names=['patient_id', 'type', 'well', 'error',
                                               'HRP2_pg_ml', 'LDH_Pan_pg_ml',
                                               'LDH_Pv_pg_ml', 'LDH_Pf_pg_ml',
                                               'CRP_ng_ml'])
    plex_data = plex_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    plex_data['patient_id'] = plex_data['patient_id'].fillna(method='ffill')
    plex_data = plex_data[~plex_data['patient_id'].isnull()]
#     test = plex_data.loc[plex_data['patient_id'].str.contains('pa-001')]
#     if len(test) > 1: 
#         print(fname)
    dfs.append(plex_data)
combined = pd.concat(dfs)
combined = combined.loc[~combined['patient_id'].str.contains('ctrl')]
combined = combined.loc[~combined['type'].isnull()]
combined = combined.loc[~combined['type'].str.contains('replicate')]
combined.head()

,patient_id,type,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml
15,qdm 1434 2019/03/07 edta wb (neat),reduced concentration,c5,NaN,< 0.68,41.02,28.01,< 5.08,> 38000.00
16,qdm 1434 2019/03/07 edta wb (1:20),reduced concentration,d5,NaN,< 13.60,< 327.20,< 93.80,< 101.60,87928.67
17,qdm 1479 2019/04/10 edta plas (neat),reduced concentration,c4,NaN,5.14,962.33,526.7,15.76,> 38000.00
18,qdm 1479 2019/04/10 edta plas (1:20),reduced concentration,d4,NaN,< 13.60,1056.63,350.9,< 101.60,> 760000.00
19,qem 31 2010/10/19 pfp (neat),reduced concentration,c2,NaN,6.26,30554.78,19174.42,60.81,> 38000.00


In [3]:
def fix_concentrations(df):
    con = df['concentration'].partition(':')[2]
    con = con.partition(')')[0]
    if len(con) != 0:
        return con
    else:
        return '1'

In [4]:
samples_data = combined.copy(deep=True)
samples_data = samples_data.drop('type', axis=1)
samples_data['concentration'] = samples_data['patient_id'].apply(lambda x: x.split(' ')[-1])
samples_data['patient_id'] = samples_data['patient_id'].apply(lambda x: '_'.join(x.split(' ')[:-1]).replace('/', '_'))
samples_data = samples_data.loc[(samples_data['concentration'].str.contains('neat|20'))]
samples_data = samples_data.loc[~samples_data['well'].isnull()]
samples_data['concentration'] = samples_data.apply(fix_concentrations, axis=1)
samples_data = samples_data.sort_values(['patient_id', 'concentration'])
samples_data.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
39,kk_103_2014_04_09_hep_plas,e11,NaN,6.52,66763.83,17747.82,81.37,> 38000.00,1
40,kk_103_2014_04_09_hep_plas,f11,NaN,23.05,82537.55,43492.8,227.52,> 760000.00,20
42,kk_103_2014_04_09_wblamp,g8,NaN,3.02,> 67000.00,19185.53,71.94,> 38000.00,1
43,kk_103_2014_04_09_wblamp,h8,NaN,< 13.60,1563.69,952.99,< 101.60,46420.38,20
41,kk_107_2014_04_15_hep_plas,e12,NaN,2.82,900.77,479.96,22.38,> 38000.00,1


In [5]:
samples_data['concentration'].unique()

array(['1', '20'], dtype=object)

In [6]:
sample_ids = samples_data['patient_id'].unique().tolist()
sample_set = set(sample_ids)

In [7]:
# threshold values for various analytes
thresholds = {'HRP2_pg_ml': 2800, 'LDH_Pan_pg_ml': 67000,
               'LDH_Pv_pg_ml': 19200, 'LDH_Pf_pg_ml': 20800,
               'CRP_ng_ml': 38000}

In [8]:
# constant to apply to the threshold for different dilutions
dil_constants = {'20': 1}

In [9]:
# positivity thresholds for various analytes
pos_thresholds = {'HRP2_pg_ml': 2.3, 'LDH_Pan_pg_ml': 47.8,
                   'LDH_Pv_pg_ml': 75.1, 'CRP_ng_ml': np.nan}

In [10]:
no_duplicates = samples_data.drop_duplicates(subset=['patient_id', 'concentration'], keep=False)
no_duplicates.head()

,patient_id,well,error,HRP2_pg_ml,LDH_Pan_pg_ml,LDH_Pv_pg_ml,LDH_Pf_pg_ml,CRP_ng_ml,concentration
39,kk_103_2014_04_09_hep_plas,e11,NaN,6.52,66763.83,17747.82,81.37,> 38000.00,1
40,kk_103_2014_04_09_hep_plas,f11,NaN,23.05,82537.55,43492.8,227.52,> 760000.00,20
42,kk_103_2014_04_09_wblamp,g8,NaN,3.02,> 67000.00,19185.53,71.94,> 38000.00,1
43,kk_103_2014_04_09_wblamp,h8,NaN,< 13.60,1563.69,952.99,< 101.60,46420.38,20
41,kk_107_2014_04_15_hep_plas,e12,NaN,2.82,900.77,479.96,22.38,> 38000.00,1


In [11]:
for err in no_duplicates['error'].unique():
    print(err)
    print(type(err))

nan
<class 'float'>


In [12]:
def return_decisions(low, high, fail='fail', weird='***'):
    # Columns = neat: [real #, LLQ, ULQ or within 10% ULQ, NA]
    # Rows = dilution: [real #, LLQ, ULQ or within 10% ULQ, NA]
    HRP2_matrix = np.array([[weird, low, high, fail],
                            [low, low, fail, fail],
                            [fail, fail, high, fail],
                            [fail, fail, fail, fail]])
    other_matrix = np.array([[weird, low, high, fail],
                            [low, low, fail, fail],
                            [fail, fail, high, fail],
                            [fail, fail, fail, fail]])
    # decisions for various analytes
    decisions = {'HRP2_pg_ml': HRP2_matrix, 'LDH_Pan_pg_ml': other_matrix,
                 'LDH_Pv_pg_ml': other_matrix, 'LDH_Pf_pg_ml': other_matrix,
                 'CRP_ng_ml': other_matrix}
    return(decisions)

In [13]:
def run_compare(df, analyte_val, dil_val):
    real, LLQ, ULQ, NA = False, False, False, False
    val = df[analyte_val]
    thresh_val = dil_constants[dil_val] * thresholds[analyte_val]
    try:
        float_val = float(val)
        if math.isnan(float_val):
            NA = True
        elif float_val > .90*thresh_val:
            ULQ = True
        else:
            real = True
    except ValueError:
        if '<' in val:
            LLQ = True
        elif '>' in val:
            ULQ = True
    finally:
        return(np.array([real, LLQ, ULQ, NA]))

In [35]:
analyte_dfs = []
error_pids = {}
for analyte in thresholds.keys():
# for analyte in ['HRP2_pg_ml']:
    print(analyte)
    patient_dfs = []
    for pid in no_duplicates['patient_id'].unique():
#     for pid in ['pa-001-3', 'pa-014-63', 'pa-129-14']:
        patient_data = no_duplicates.loc[no_duplicates['patient_id'] == pid]
        dilution_values = sorted([val for val in patient_data['concentration'].unique() if val != '1'], key=len)
        best_decision = '1'
        for max_dilution in dilution_values:
#         for max_dilution in ['15625000000']:
            dil_data = patient_data.loc[patient_data['concentration'].isin([best_decision, max_dilution])]          
            partial_compare = partial(run_compare, analyte_val=analyte, dil_val=max_dilution)
            dil_data['decision_vector'] = dil_data.apply(partial_compare, axis=1)
            decisions = return_decisions(best_decision, max_dilution)
            decision_matrix = decisions[analyte]
            best_df = pd.DataFrame(columns=['patient_id', 'errors', analyte,
                                            '{}_dilution'.format(analyte),
                                            '{}_well'.format(analyte),
#                                             '{}_max_dilution'.format(analyte),
                                            '{}_neat_val'.format(analyte),
                                            '{}_neat_percent_ULQ'.format(analyte),
                                            '{}_20_val'.format(analyte),
                                            '{}_20_percent_ULQ'.format(analyte)])
            vector_low = dil_data.loc[dil_data['concentration'] == best_decision,
                                      'decision_vector'].item()
            vector_high = dil_data.loc[dil_data['concentration'] == max_dilution,
                                       'decision_vector'].item()
            decision = decision_matrix[vector_high, vector_low].item()
            if decision in [best_decision, max_dilution]:
                val = dil_data.loc[dil_data['concentration'] == decision,
                                   analyte].item()
                well = dil_data.loc[dil_data['concentration'] == decision,
                                    'well'].item()
                error = dil_data.loc[dil_data['concentration'] == decision,
                                    'error'].item()
                val_for = dil_data.loc[dil_data['concentration'] == '1',
                                   analyte].item()
                try:
                    val_per_ULQ = 100 * (float(val_for) / (thresholds[analyte]))
                except ValueError:
                    val_per_ULQ = np.nan
                val_20_for = dil_data.loc[dil_data['concentration'] == '20',
                                   analyte].item()
                try:
                    val_20_per_ULQ = 100 * (float(val_20_for) / (20 * thresholds[analyte]))
                except ValueError:
                    val_20_per_ULQ = np.nan
            elif decision == '***':
                val = '***'
                val_for = dil_data.loc[dil_data['concentration'] == '1',
                                   analyte].item()
                val_per_ULQ = 100 * (float(val_for) / (thresholds[analyte]))
                val_20_for = dil_data.loc[dil_data['concentration'] == '20',
                                   analyte].item()
                val_20_per_ULQ = 100 * (float(val_20_for) / (20 * thresholds[analyte]))
                well = dil_data.loc[dil_data['concentration'] == '1',
                                    'well'].item()
                error = np.nan
                error_pids[pid] = 'Multiple real #s'
            elif decision == 'fail':
                val = 'fail'
                well = 'fail'
                error = np.nan
                val_for = np.nan
                val_20_for = np.nan
                val_per_ULQ = np.nan
                val_20_per_ULQ = np.nan
                error_pids[pid] = '{} failure'.format(analyte)
            else:
                raise ValueError("Unexpected decision value: {}".format(decision))
            other_dilutions = [val for val in patient_data['concentration'].unique()]
            other_dilutions = [float(val) for val in other_dilutions if val not in ('fail', '***')]
            max_dilution = int(max(other_dilutions))
            df_decision = decision if decision not in ('fail', '***') else np.nan
            best_decision = decision
            best_df = best_df.append({'patient_id': pid, 'errors': error, analyte: val,
                                      '{}_dilution'.format(analyte): df_decision,
                                      '{}_well'.format(analyte): well,
#                                       '{}_max_dilution'.format(analyte): max_dilution,
                                      '{}_neat_val'.format(analyte): val_for,
                                      '{}_neat_percent_ULQ'.format(analyte): val_per_ULQ,
                                      '{}_20_val'.format(analyte): val_20_for,
                                      '{}_20_percent_ULQ'.format(analyte): val_20_per_ULQ}, ignore_index=True)
            if decision == 'fail':
                break
            if decision == '***':
                break
        patient_dfs.append(best_df)
    patient_df = pd.concat(patient_dfs)
    patient_df['errors'] = patient_df['errors'].astype('object')
    analyte_dfs.append(patient_df)
final_df = reduce(lambda left, right: pd.merge(left, right, on=['patient_id', 'errors']), analyte_dfs)

HRP2_pg_ml


C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:54: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:57: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:60: FutureWarning: `item` has been deprecated and will be removed in a future version
C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: FutureWa

LDH_Pan_pg_ml
LDH_Pv_pg_ml
LDH_Pf_pg_ml
CRP_ng_ml


In [36]:
super_final = final_df.copy(deep=True)
for pid in error_pids.keys():
    error = error_pids[pid]
    pid_df = final_df.loc[final_df['patient_id'] == pid]
    pid_df['errors'] = pid_df['errors'].apply(lambda x: error if np.isnan(x) else x + ' ' + error)
    if len(pid_df) > 0:
        super_final = super_final.loc[super_final['patient_id'] != pid]
        super_final = super_final.append(pid_df)
print(len(super_final))
super_final.tail()

C:\Users\lzoeckler\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


504


,patient_id,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_neat_percent_ULQ,HRP2_pg_ml_20_val,HRP2_pg_ml_20_percent_ULQ,LDH_Pan_pg_ml,...,LDH_Pf_pg_ml_neat_percent_ULQ,LDH_Pf_pg_ml_20_val,LDH_Pf_pg_ml_20_percent_ULQ,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_neat_percent_ULQ,CRP_ng_ml_20_val,CRP_ng_ml_20_percent_ULQ
323,qem_129_2011_02_27_pfp,Multiple real #s,1.88,1,g7,1.88,0.067143,< 13.60,NaN,< 16.36,...,0.055865,110.95,0.026671,68767.81,20,h7,> 38000.00,NaN,68767.81,9.048396
486,qes_44_2014_04_03_pfp,Multiple real #s,0.7,1,a12,0.7,0.025000,< 13.60,NaN,< 16.36,...,0.053894,156.56,0.037635,235502.85,20,b12,> 38000.00,NaN,235502.85,30.987217
169,qdm_1634_2019_07_15_edta_wb,Multiple real #s,2.75,1,g8,2.75,0.098214,< 13.60,NaN,< 16.36,...,NaN,< 101.60,NaN,***,NaN,g8,3786.6,9.964737,2644.94,0.348018
340,qem_150_2011_03_22_pfp,CRP_ng_ml failure,5.98,1,c8,5.98,0.213571,< 13.60,NaN,39.47,...,0.110817,< 101.60,NaN,fail,NaN,fail,NaN,NaN,NaN,NaN
381,qem_205_2011_05_06_pfp,CRP_ng_ml failure,> 56000.00,20,h12,> 2800.00,NaN,> 56000.00,NaN,106891.68,...,NaN,145979.26,35.091168,fail,NaN,fail,NaN,NaN,NaN,NaN


In [37]:
super_final['errors'].unique()

array([nan, 'Multiple real #s', 'HRP2_pg_ml failure',
       'LDH_Pan_pg_ml failure', 'LDH_Pv_pg_ml failure',
       'LDH_Pf_pg_ml failure', 'CRP_ng_ml failure'], dtype=object)

In [38]:
# final_ids = super_final['patient_id'].unique().tolist()
# final_set = set(final_ids)
# assert final_set == sample_set, print(sample_set - final_set)

In [39]:
def split_time(df, run):
    if run:
        sub = df['patient_id'].split('_')
        try:
            time = '_'.join(sub[2:])
            return time
        except IndexError:
            return 0
    else: raise KeyError('BUSTOOOO')

In [40]:
def remove_time(df):
    patient = df['patient_id'].split('_')
    return '_'.join(patient[0:2])

In [41]:
time_df = super_final.copy(deep=True)
# time_df['date'] = time_df.apply(split_time, axis=1, run=True)
# time_df['patient_id'] = time_df.apply(remove_time, axis=1)
time_df.sort_values(['patient_id'], inplace=True)
time_df.set_index(['patient_id'], inplace=True)
time_df.head()

,errors,HRP2_pg_ml,HRP2_pg_ml_dilution,HRP2_pg_ml_well,HRP2_pg_ml_neat_val,HRP2_pg_ml_neat_percent_ULQ,HRP2_pg_ml_20_val,HRP2_pg_ml_20_percent_ULQ,LDH_Pan_pg_ml,LDH_Pan_pg_ml_dilution,...,LDH_Pf_pg_ml_neat_percent_ULQ,LDH_Pf_pg_ml_20_val,LDH_Pf_pg_ml_20_percent_ULQ,CRP_ng_ml,CRP_ng_ml_dilution,CRP_ng_ml_well,CRP_ng_ml_neat_val,CRP_ng_ml_neat_percent_ULQ,CRP_ng_ml_20_val,CRP_ng_ml_20_percent_ULQ
patient_id,,,,,,,,,,,,,,,,,,,,,
kk_103_2014_04_09_hep_plas,Multiple real #s,***,NaN,e11,6.52,0.232857,23.05,0.041161,82537.55,20,...,0.391202,227.52,0.054692,> 760000.00,20,f11,> 38000.00,NaN,> 760000.00,NaN
kk_103_2014_04_09_wblamp,NaN,3.02,1,g8,3.02,0.107857,< 13.60,NaN,1563.69,20,...,0.345865,< 101.60,NaN,46420.38,20,h8,> 38000.00,NaN,46420.38,6.107945
kk_107_2014_04_15_hep_plas,Multiple real #s,***,NaN,e12,2.82,0.100714,13.99,0.024982,***,NaN,...,0.107596,156.56,0.037635,67851.5,20,f12,> 38000.00,NaN,67851.5,8.927829
kk_107_2014_04_15_wblamp,Multiple real #s,2.12,1,g9,2.12,0.075714,< 13.60,NaN,fail,NaN,...,0.051442,385.06,0.092563,48678.79,20,h9,> 38000.00,NaN,48678.79,6.405104
kk_113_2014_05_12_hep_plas,Multiple real #s,0.93,1,c11,0.93,0.033214,< 13.60,NaN,218.1,1,...,0.040913,142.87,0.034344,85202.89,20,d11,> 38000.00,NaN,85202.89,11.210907


In [42]:
time_df.to_csv('C:/Users/lzoeckler/Desktop/new_dilutions.csv')

In [142]:
partial_format = samples_data.copy(deep=True)
partial_format['time_point_days'] = partial_format.apply(split_time, axis=1)
partial_format['patient_id'] = partial_format.apply(remove_time, axis=1)
partial_format = partial_format.merge(add_info, how='left', on=['patient_id', 'time_point_days'])
partial_format.sort_values(['patient_id', 'time_point_days'], inplace=True)
partial_format.set_index(['patient_id', 'time_point_days'], inplace=True)
partial_format.head()

well error     HRP2_pg_ml    LDH_Pan_pg_ml  \
patient_id time_point_days                                              
pa-001     3                 g6   NaN         145.95           142.30   
           3                 f3   NaN      397468.45     < 1801250.00   
           3                 e3   NaN      318702.59         74360.14   
           3                 h3   NaN  1537195777.01  119019730410.15   
           3                 h6   NaN       10273.61          2535.11   

                              LDH_Pv_pg_ml        CRP_ng_ml concentration  \
patient_id time_point_days                                                  
pa-001     3                         15.35          5678.58             1   
           3                   < 291250.00     < 3342500.00        125000   
           3                      10385.14       < 66850.00          2500   
           3                16185275422.09  286153421264.94     312500000   
           3                        733.44         49491.76            50   

                            day0_enrollee  days_since_tx  initial_sample  \
patient_id time_point_days                                                 
pa-001     3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   
           3                          1.0            3.0             NaN   

                            got_PQ  age     sex  fever48_r    enrolled_from  \
patient_id time_point_days                                                    
pa-001     3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   
           3                   1.0  2.0  female        1.0  health facility   

                            returned_with_fever when_returned_with_fever  \
patient_id time_point_days                                                 
pa-001     3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            
           3                                NaN                            

                            retreated when_retreated  
patient_id time_point_days                            
pa-001     3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN                 
           3                      NaN

In [143]:
partial_format.to_csv('C:/Users/lzoeckler/Desktop/4plex/output_data/partially_formatted.csv')